In [20]:
import os
import pickle
import pandas as pd
import numpy as np
import itertools
from lendingclub import config

pd.set_option('max_columns', 999)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
np.set_printoptions(threshold=np.inf)

In [2]:
csv_path = config.wrk_csv_dir
# for now its always been one csv. Will have to revisit if they break it out to multiple
pmt_hist_fnames = [f for f in os.listdir(csv_path) if 'PMTHIST' in f]
if len(pmt_hist_fnames) > 1:
    sys.exit('more than one payment history file')

# chunking for smaller development set
print('chunking for smaller dev set')
with open(os.path.join(config.data_dir, 'dev_ids.pkl'), "rb") as input_file:
    dev_ids = pickle.load(input_file)

reader = pd.read_csv(os.path.join(csv_path, pmt_hist_fnames[0]), chunksize=25000000, iterator=True)# 25000000 best so far
pmt_hist = pd.concat([chunk.query('LOAN_ID in @dev_ids') for chunk in reader])

chunking for smaller dev set


/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# examining loading in 04_clean_pmt_hist

In [68]:
csv_path = config.wrk_csv_dir
# for now its always been one csv. Will have to revisit if they break it out to multiple
pmt_hist_fnames = [f for f in os.listdir(csv_path) if 'PMTHIST' in f]
if len(pmt_hist_fnames) > 1:
    sys.exit('more than one payment history file, need to change this code')

# load in dev_ids.pkl and pmt_hist_skiprows
with open(os.path.join(config.data_dir, 'dev_ids.pkl'), "rb") as f:
    dev_ids = pickle.load(f)
with open(os.path.join(config.data_dir, 'pmt_hist_skiprows.pkl'), "rb") as f:
    pmt_hist_skiprows = pickle.load(f)

print('loading pmt_hist; skipping {0} rows'.format(len(pmt_hist_skiprows)))
pmt_hist = pd.read_csv(os.path.join(csv_path, pmt_hist_fnames[0]), skiprows=pmt_hist_skiprows, header=0)
print(pmt_hist.columns)

loading pmt_hist; skipping 50282328 rows
Index(['114408', '5766.5463849', '0', '0.1', '0.2', '1535.56', '0.3',
       'Unnamed: 7', 'Charged Off', 'JUN2009', '5766.5463849.1', '22', '1',
       '5766.5463849.2', '0.1691', 'AUG2007', '284.86', '20.22', 'FL', 'OWN',
       '5000', 'JUL1990', '5', '23', '6755', '0.993', '1.1', '0.4', '33',
       '0.5', '0.6', '10+ years', '0.7', 'G', '36', '645-649', 'LOW-499',
       '07Q3', '0.55', '0.8'],
      dtype='object')


/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
pmt_hist_skiprows.remove(0)

In [72]:
0 in pmt_hist_skiprows

False

In [73]:
pmt_hist = pd.read_csv(os.path.join(csv_path, pmt_hist_fnames[0]), skiprows=pmt_hist_skiprows, header=0)

/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
pmt_hist.head()

,LOAN_ID,PBAL_BEG_PERIOD,PRNCP_PAID,INT_PAID,FEE_PAID,DUE_AMT,RECEIVED_AMT,RECEIVED_D,PERIOD_END_LSTAT,MONTH,PBAL_END_PERIOD,MOB,CO,COAMT,InterestRate,IssuedDate,MONTHLYCONTRACTAMT,dti,State,HomeOwnership,MonthlyIncome,EarliestCREDITLine,OpenCREDITLines,TotalCREDITLines,RevolvingCREDITBalance,RevolvingLineUtilization,Inquiries6M,DQ2yrs,MonthsSinceDQ,PublicRec,MonthsSinceLastRec,EmploymentLength,currentpolicy,grade,term,APPL_FICO_BAND,Last_FICO_BAND,VINTAGE,PCO_RECOVERY,PCO_COLLECTION_FEE
0,114408,5766.546385,0.000000,0.000000,0.0,1535.56,0.00,NaN,Charged Off,JUN2009,5766.546385,22,1,5766.546385,0.1691,AUG2007,284.86,20.22,FL,OWN,5000.000000,JUL1990,5,23,6755,0.993,1,0,33.0,0,0.0,10+ years,0,G,36,645-649,LOW-499,07Q3,0.55,0.0
1,114469,13000.000000,278.339083,186.600917,0.0,464.95,464.94,SEP2007,Current,SEP2007,12721.660917,1,0,0.000000,0.1722,AUG2007,464.95,15.98,FL,RENT,2666.666667,MAY2005,6,7,6703,0.798,1,0,0.0,0,0.0,1 year,0,G,36,640-644,640-644,07Q3,NaN,NaN
2,114469,12721.660917,282.334339,182.605661,0.0,464.95,464.94,OCT2007,Current,OCT2007,12439.326577,2,0,0.000000,0.1722,AUG2007,464.95,15.98,FL,RENT,2666.666667,MAY2005,6,7,6703,0.798,1,0,0.0,0,0.0,1 year,0,G,36,640-644,MISSING,07Q3,NaN,NaN
3,114469,12439.326577,286.386943,178.553057,0.0,464.95,464.94,NOV2007,Current,NOV2007,12152.939634,3,0,0.000000,0.1722,AUG2007,464.95,15.98,FL,RENT,2666.666667,MAY2005,6,7,6703,0.798,1,0,0.0,0,0.0,1 year,0,G,36,640-644,MISSING,07Q3,NaN,NaN
4,114469,12152.939634,290.497717,174.442283,0.0,464.95,464.94,DEC2007,Current,DEC2007,11862.441917,4,0,0.000000,0.1722,AUG2007,464.95,15.98,FL,RENT,2666.666667,MAY2005,6,7,6703,0.798,1,0,0.0,0,0.0,1 year,0,G,36,640-644,MISSING,07Q3,NaN,NaN


# examing line 147 from 04_clean_pmt_hist

In [60]:
raw_pmt_history = pd.read_csv(os.path.join(config.data_dir, 'csvs/02_working_csvs/PMTHIST_ALL_201909.csv'))
raw_loan_info = pd.read_feather(os.path.join(config.data_dir, 'raw_loan_info.fth'))
base_loan_info = pd.read_feather(os.path.join(config.data_dir, 'base_loan_info.fth'))
pmt_history = pd.read_feather(os.path.join(config.data_dir, 'clean_pmt_history.fth'))
print(base_loan_info.shape, pmt_history.shape)

/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2507335, 90) (50371978, 40)


In [61]:
# examine id 12287070 and see what could be wrong with its rate?
id_ex = 12287070
np.pmt(.06/12, 60, -27600)

533.5853222122203

In [62]:
raw_loan_info.query('id == @id_ex')

,id,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
1139845,12287070,E,E2,Principal Coordinator,10+ years,MORTGAGE,Source Verified,Feb-2014,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,None,debt_consolidation,Debt consolidation,232xx,VA,Oct-1994,69.1%,w,Jan-2018,None,Aug-2019,Individual,None,None,N,None,None,None,None,None,None,None,N,None,None,None,27600.0,27600.0,27600.0,60,6.0,614.299988,75000.0,22.08,0.0,675.0,679.0,3.0,NaN,NaN,17.0,0.0,23483.0,47.0,0.0,0.0,41594.644531,41594.640625,27600.0,13957.700195,36.939999,0.0,0.0,8850.799805,689.0,685.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0,243242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34000.0,NaN,NaN,NaN,9.0,15203.0,3774.0,85.699997,0.0,0.0,121.0,232.0,6.0,5.0,2.0,36.0,NaN,3.0,NaN,0.0,5.0,6.0,5.0,10.0,22.0,10.0,23.0,6.0,17.0,0.0,0.0,0.0,5.0,100.0,60.0,0.0,0.0,264063.0,80760.0,26300.0,66675.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
raw_pmt_history.query('LOAN_ID == @id_ex')

,LOAN_ID,PBAL_BEG_PERIOD,PRNCP_PAID,INT_PAID,FEE_PAID,DUE_AMT,RECEIVED_AMT,RECEIVED_D,PERIOD_END_LSTAT,MONTH,PBAL_END_PERIOD,MOB,CO,COAMT,InterestRate,IssuedDate,MONTHLYCONTRACTAMT,dti,State,HomeOwnership,MonthlyIncome,EarliestCREDITLine,OpenCREDITLines,TotalCREDITLines,RevolvingCREDITBalance,RevolvingLineUtilization,Inquiries6M,DQ2yrs,MonthsSinceDQ,PublicRec,MonthsSinceLastRec,EmploymentLength,currentpolicy,grade,term,APPL_FICO_BAND,Last_FICO_BAND,VINTAGE,PCO_RECOVERY,PCO_COLLECTION_FEE
7293654,12287070,27600.000000,267.510000,471.270000,0.00,738.78,738.78,APR2014,Current,MAR2014,27332.490000,1,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,700-704,14Q1,NaN,NaN
7293655,12287070,27332.490000,272.077733,466.702267,0.00,738.78,738.78,MAY2014,Current,MAY2014,27060.412267,2,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,750-754,14Q1,NaN,NaN
7293656,12287070,27060.412267,276.723461,462.056539,0.00,738.78,738.78,JUN2014,Current,MAY2014,26783.688806,3,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,740-744,14Q1,NaN,NaN
7293657,12287070,26783.688806,281.448514,457.331486,0.00,738.78,738.78,JUL2014,Current,JUL2014,26502.240293,4,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,700-704,14Q1,NaN,NaN
7293658,12287070,26502.240293,286.254247,452.525753,0.00,738.78,738.78,AUG2014,Current,AUG2014,26215.986046,5,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,695-699,14Q1,NaN,NaN
7293659,12287070,26215.986046,291.142038,447.637962,0.00,738.78,738.78,SEP2014,Current,SEP2014,25924.844007,6,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,695-699,14Q1,NaN,NaN
7293660,12287070,25924.844007,296.113289,442.666711,0.00,738.78,738.78,OCT2014,Current,OCT2014,25628.730719,7,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,665-669,14Q1,NaN,NaN
7293661,12287070,25628.730719,301.169423,437.610577,0.00,738.78,738.78,NOV2014,Current,NOV2014,25327.561296,8,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,705-709,14Q1,NaN,NaN
7293662,12287070,25327.561296,306.311891,432.468109,0.00,738.78,738.78,DEC2014,Current,DEC2014,25021.249405,9,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,685-689,14Q1,NaN,NaN
7293663,12287070,25021.249405,311.542166,427.237834,0.00,738.78,738.78,JAN2015,Current,JAN2015,24709.707238,10,0,0.0,0.06,FEB2014,614.3,22.08,VA,MORTGAGE,6250.0,OCT1994,17.0,47.0,23483,0.691,3,0.0,NaN,0,NaN,10+ years,1,E,60,675-679,690-694,14Q1,NaN,NaN


In [64]:
base_loan_info.query('id == @id_ex')

,sec_app_collections_12_mths_ex_med,pct_tl_nvr_dlq,sub_grade,grade,total_acc,sec_app_open_acc,fico_range_low,all_util,sec_app_open_act_il,num_bc_sats,open_il_24m,num_actv_bc_tl,dti,inq_last_12m,revol_bal_joint,num_rev_accts,mths_since_recent_revol_delinq,num_tl_30dpd,sec_app_mort_acc,sec_app_chargeoff_within_12_mths,total_cu_tl,num_il_tl,id,int_rate,total_bal_ex_mort,sec_app_num_rev_accts,tax_liens,num_rev_tl_bal_gt_0,sec_app_earliest_cr_line,annual_inc,home_ownership,addr_state,application_type,num_bc_tl,max_bal_bc,sec_app_fico_range_low,mths_since_recent_inq,collections_12_mths_ex_med,annual_inc_joint,percent_bc_gt_75,mths_since_last_major_derog,mths_since_rcnt_il,bc_open_to_buy,mo_sin_old_il_acct,num_tl_op_past_12m,sec_app_mths_since_last_major_derog,term,purpose,open_rv_24m,avg_cur_bal,inq_fi,open_rv_12m,revol_bal,total_rev_hi_lim,num_tl_90g_dpd_24m,tot_hi_cred_lim,tot_cur_bal,initial_list_status,mths_since_last_record,num_sats,tot_coll_amt,num_op_rev_tl,bc_util,mths_since_recent_bc_dlq,delinq_amnt,pub_rec,sec_app_fico_range_high,fico_range_high,mths_since_last_delinq,mo_sin_old_rev_tl_op,dti_joint,mort_acc,mo_sin_rcnt_rev_tl_op,open_acc_6m,total_bc_limit,chargeoff_within_12_mths,mths_since_recent_bc,sec_app_revol_util,acc_now_delinq,total_il_high_credit_limit,revol_util,pub_rec_bankruptcies,open_act_il,open_acc,num_actv_rev_tl,total_bal_il,mo_sin_rcnt_tl,earliest_cr_line,emp_length,open_il_12m
262430,NaN,1.0,E2,E,47.0,NaN,675.0,NaN,NaN,5.0,NaN,5.0,0.2208,NaN,NaN,23.0,NaN,0.0,NaN,NaN,NaN,22.0,12287070,0.06,80760.0,NaN,0.0,6.0,NaT,75000.0,mortgage,VA,individual,10.0,NaN,NaN,3.0,0.0,NaN,0.6,NaN,NaN,3774.0,121.0,5.0,NaN,60,debt_consolidation,NaN,15203.0,NaN,NaN,23483.0,34000.0,0.0,264063.0,243242.0,w,NaN,17.0,0.0,10.0,0.857,NaN,0.0,0.0,NaN,679.0,NaN,232.0,NaN,2.0,6.0,NaN,26300.0,0.0,36.0,NaN,0.0,66675.0,0.691,0.0,NaN,17.0,6.0,NaN,5.0,1994-10-01,10+ years,NaN


In [65]:
pmt_history.query('loan_id == @id_ex')

,pmt_date,status_period_end,date,issue_d,addr_state,home_ownership,first_credit_line,emp_len,grade,vintage,outs_princp_beg,outs_princp_end,m_on_books,int_rate,monthly_pmt,dti,m_income,open_credit_lines,total_credit_lines,revol_line_util,dq_24m,m_since_dq,m_since_rec,current_policy,term,public_recs,fico_apply,fico_last,loan_id,revol_credit_bal,princp_paid,int_paid,fee_paid,amt_due,amt_paid,charged_off_this_month,charged_off_amt,recovs,recov_fees,all_cash_to_inv
7292889,2014-04-01,current,2014-03-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,27600.000000,27332.490234,1,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,702,12287070,23483.0,267.510010,471.269989,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
7292890,2014-04-01,current,2014-04-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,27600.000000,27332.490234,1,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,702,12287070,23483.0,267.510010,471.269989,0.000000,738.780029,0.000000,0.0,0.0,0.0,0.0,738.780029
7292891,2014-06-01,current,2014-05-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,27332.490234,26783.689453,3,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,742,12287070,23483.0,548.801025,928.759033,0.000000,1477.560059,1477.560059,0.0,0.0,0.0,0.0,1477.560059
7292892,2014-06-01,current,2014-06-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,27332.490234,26783.689453,3,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,742,12287070,23483.0,548.801025,928.759033,0.000000,1477.560059,0.000000,0.0,0.0,0.0,0.0,1477.560059
7292893,2014-07-01,current,2014-07-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,26783.689453,26502.240234,4,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,702,12287070,23483.0,281.449005,457.330994,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
7292894,2014-08-01,current,2014-08-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,26502.240234,26215.986328,5,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,697,12287070,23483.0,286.253998,452.526001,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
7292895,2014-09-01,current,2014-09-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,26215.986328,25924.843750,6,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,697,12287070,23483.0,291.141998,447.638000,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
7292896,2014-10-01,current,2014-10-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,25924.843750,25628.730469,7,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,667,12287070,23483.0,296.113007,442.666992,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
7292897,2014-11-01,current,2014-11-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,25628.730469,25327.560547,8,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,707,12287070,23483.0,301.169006,437.610992,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
7292898,2014-12-01,current,2014-12-01,2014-02-01,VA,mortgage,1994-10-01,10+ years,E,14Q1,25327.560547,25021.250000,9,0.06,614.299988,22.08,6250.0,17.0,47.0,0.691,0.0,NaN,NaN,1,60,0.0,677,687,12287070,23483.0,306.312012,432.467987,0.000000,738.780029,738.780029,0.0,0.0,0.0,0.0,738.780029
